In [1]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, initialize_env
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import pytz

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
df = pd.read_excel('Coupon_testing.xlsx')
df

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,RETAILER_ID,ACTIVE_SELLERS,DAYS_INACTIVE,NMV_LAST_45,NMV,ORDER_COUNT_LAST_45,ORDER_COUNT,random,Counter,group_type
0,147884,27,11,11937.00000,327817.134038,6,97,0.000006,0,C1
1,349281,24,15,10405.00000,237968.068788,4,108,0.000148,1,C1
2,714314,26,43,1711.75000,24631.277662,1,10,0.000579,2,C1
3,360479,27,9,9724.75000,106049.359650,5,70,0.000708,3,C1
4,268508,37,17,36717.94000,548429.627536,5,92,0.000985,4,C1
...,...,...,...,...,...,...,...,...,...,...
4932,606793,32,27,7904.50000,27819.459000,3,17,0.998693,4932,T4
4933,286448,32,28,2106.55000,158641.149620,1,62,0.998869,4933,T4
4934,926882,7,28,1799.96875,1799.968750,1,1,0.998990,4934,T4
4935,475396,38,22,7372.50000,47202.867800,2,27,0.999208,4935,T4


In [6]:
coupon_data = df[df['group_type'].isin(['T3', 'T4'])]
coupon_data

,RETAILER_ID,ACTIVE_SELLERS,DAYS_INACTIVE,NMV_LAST_45,NMV,ORDER_COUNT_LAST_45,ORDER_COUNT,random,Counter,group_type
2906,594978,16,19,1296.000000,15344.081200,1,12,0.600025,2906,T3
2907,619170,17,8,20104.019333,192284.997733,7,58,0.600333,2907,T3
2908,17311,35,11,11914.350000,328468.518870,3,105,0.600390,2908,T3
2909,245132,39,8,4757.750000,63113.580308,3,47,0.600758,2909,T3
2910,63547,24,16,5073.250000,347440.177887,1,135,0.600791,2910,T3
...,...,...,...,...,...,...,...,...,...,...
4932,606793,32,27,7904.500000,27819.459000,3,17,0.998693,4932,T4
4933,286448,32,28,2106.550000,158641.149620,1,62,0.998869,4933,T4
4934,926882,7,28,1799.968750,1799.968750,1,1,0.998990,4934,T4
4935,475396,38,22,7372.500000,47202.867800,2,27,0.999208,4935,T4


In [11]:

from datetime import datetime, timedelta
# Replace with your WonderPush API key
API_KEY = "YjFkMmExOWFlN2IxNzQzYzFlMzA4YThhNmE4ZDgwZmFiNmFlNTM2ODVhODhjMzM1NjA3NWU0MWI3M2YxYTZmYw"


user_id = 'EG_retailers_299600'




# Build the PATCH body: only the field you want to change under "custom"
patch_body = {
    "custom": {

        "string_coupon_ammount": ""
    }
}

# Send a PATCH to /v1/users/{userId}
url = (
    f"https://management-api.wonderpush.com/v1/users/{user_id}"
    f"?accessToken={API_KEY}"
)
headers = {"Content-Type": "application/json"}
response = requests.patch(url, headers=headers, json=patch_body)

if response.status_code in  [200, 201, 202]:
    print(f"✅ Patched string_mpproducts for user {user_id}")
else:
    print(f"❌ Error for {user_id}: {response.status_code} – {response.text}")

    

✅ Patched string_mpproducts for user EG_retailers_299600
